In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
print("불러온 키:", os.getenv("ECOS_API_KEY"))


불러온 키: I3K6MEX76WLHVQHHG180


In [2]:
import json

# 파일 로드
with open("ecos_items.json", "r", encoding="utf-8") as f:
    items = json.load(f)

# 특정 항목 예시 출력
for name, meta in items.items():
    print(f"▶ {name}")
    print(f"  - 통계코드: {meta['stat_code']}")
    print(f"  - 주기: {meta['cycle']}")
    print(f"  - 항목코드: {meta['item_code']}\n")


▶ 실질GDP
  - 통계코드: 200Y104
  - 주기: Q
  - 항목코드: 1400

▶ 환율
  - 통계코드: 731Y002
  - 주기: M
  - 항목코드: 0000001

▶ CPI
  - 통계코드: 902Y001
  - 주기: M
  - 항목코드: 0000001

▶ PPI
  - 통계코드: 901Y001
  - 주기: M
  - 항목코드: 0000001

▶ CD_91일
  - 통계코드: 060Y001
  - 주기: M
  - 항목코드: 010101000

▶ 국고채_3Y
  - 통계코드: 060Y001
  - 주기: M
  - 항목코드: 010301000



In [3]:
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv

# ECOS 키 불러오기
load_dotenv()
api_key = os.getenv("ECOS_API_KEY")

# 수집 대상 지표 정의
indicators = {
    "실질GDP": {"stat_code": "200Y104", "cycle": "Q", "item_code": "1400"},
    "CPI": {"stat_code": "902Y001", "cycle": "M", "item_code": "0000001"},
    "PPI": {"stat_code": "901Y001", "cycle": "M", "item_code": "0000001"},
    "환율": {"stat_code": "731Y002", "cycle": "M", "item_code": "0000001"},
    "CD_91일": {"stat_code": "060Y001", "cycle": "M", "item_code": "010101000"},
    "국고채_3Y": {"stat_code": "060Y001", "cycle": "M", "item_code": "010301000"},
}

# 수집 기간
start_year = 2015
end_year = 2023

# 결과 리스트
all_data = []

# 요청 함수
def fetch_data(name, info):
    freq = info["cycle"]
    start = f"{start_year}Q1" if freq == "Q" else f"{start_year}01"
    end = f"{end_year}Q4" if freq == "Q" else f"{end_year}12"

    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/10000/{info['stat_code']}/{freq}/{start}/{end}/{info['item_code']}"
    print(f"▶ 요청 중: {name}")
    print(f"📡 {url}")
    
    try:
        res = requests.get(url, timeout=10)
        data = res.json()
        rows = data.get("StatisticSearch", {}).get("row", [])
        for r in rows:
            r["지표명"] = name
        return rows
    except Exception as e:
        print(f"❌ 오류: {e}")
        return []

# 수집 실행
for name, info in indicators.items():
    result = fetch_data(name, info)
    all_data.extend(result)

# 엑셀로 저장
if all_data:
    df = pd.DataFrame(all_data)
    df.to_excel("ecos_macro_data_2015_2023.xlsx", index=False, engine='openpyxl')
    print("💾 저장 완료: ecos_macro_data_2015_2023.xlsx")
else:
    print("⚠️ 유효한 데이터 없음")


▶ 요청 중: 실질GDP
📡 https://ecos.bok.or.kr/api/StatisticSearch/I3K6MEX76WLHVQHHG180/json/kr/1/10000/200Y104/Q/2015Q1/2023Q4/1400
▶ 요청 중: CPI
📡 https://ecos.bok.or.kr/api/StatisticSearch/I3K6MEX76WLHVQHHG180/json/kr/1/10000/902Y001/M/201501/202312/0000001
▶ 요청 중: PPI
📡 https://ecos.bok.or.kr/api/StatisticSearch/I3K6MEX76WLHVQHHG180/json/kr/1/10000/901Y001/M/201501/202312/0000001
▶ 요청 중: 환율
📡 https://ecos.bok.or.kr/api/StatisticSearch/I3K6MEX76WLHVQHHG180/json/kr/1/10000/731Y002/M/201501/202312/0000001
▶ 요청 중: CD_91일
📡 https://ecos.bok.or.kr/api/StatisticSearch/I3K6MEX76WLHVQHHG180/json/kr/1/10000/060Y001/M/201501/202312/010101000
▶ 요청 중: 국고채_3Y
📡 https://ecos.bok.or.kr/api/StatisticSearch/I3K6MEX76WLHVQHHG180/json/kr/1/10000/060Y001/M/201501/202312/010301000
💾 저장 완료: ecos_macro_data_2015_2023.xlsx


In [4]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("ECOS_API_KEY")

# 찾고자 하는 지표명 키워드
search_keywords = {
    "CPI": "소비자물가지수",
    "PPI": "생산자물가지수",
    "환율": "환율",
    "CD_91일": "CD",
    "국고채_3Y": "국고채",
}

def get_stat_list():
    url = f"https://ecos.bok.or.kr/api/StatisticTableList/{api_key}/json/kr/1/1000/"
    res = requests.get(url)
    data = res.json()
    rows = data.get("StatisticTableList", {}).get("row", [])
    return rows

def get_item_list(stat_code):
    url = f"https://ecos.bok.or.kr/api/StatItemList/{api_key}/json/kr/1/1000/{stat_code}"
    res = requests.get(url)
    data = res.json()
    return data.get("StatItemList", {}).get("row", [])

# 전체 통계표 목록 불러오기
all_tables = get_stat_list()

# 지표별 후보 추출
for key, keyword in search_keywords.items():
    print(f"\n🔍 [{key}] 관련 통계표 검색어: '{keyword}'")

    filtered = [t for t in all_tables if keyword in t.get("STAT_NAME", "")]

    if not filtered:
        print("❌ 관련 통계표 없음")
        continue

    for table in filtered:
        print(f"📌 코드: {table['STAT_CODE']}, 이름: {table['STAT_NAME']}")

        # 항목코드 확인
        items = get_item_list(table['STAT_CODE'])
        print(f"  └ 항목코드 {len(items)}개:")
        for i in items[:5]:  # 처음 5개만 미리보기
            print(f"    - {i['ITEM_CODE']} : {i['ITEM_NAME']}")



🔍 [CPI] 관련 통계표 검색어: '소비자물가지수'
📌 코드: 0000000211, 이름: 4.2. 소비자물가지수(2020=100)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [5]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("ECOS_API_KEY")

search_keywords = {
    "CPI": "소비자물가지수",
    "PPI": "생산자물가지수",
    "환율": "환율",
    "CD_91일": "CD",
    "국고채_3Y": "국고채",
}

def get_stat_list():
    url = f"https://ecos.bok.or.kr/api/StatisticTableList/{api_key}/json/kr/1/1000/"
    res = requests.get(url)
    try:
        data = res.json()
    except Exception as e:
        print("❌ [통계표 요청 실패] 응답이 JSON 형식이 아님")
        print(res.text)
        return []
    return data.get("StatisticTableList", {}).get("row", [])

def get_item_list(stat_code):
    url = f"https://ecos.bok.or.kr/api/StatItemList/{api_key}/json/kr/1/1000/{stat_code}"
    res = requests.get(url)
    try:
        data = res.json()
    except Exception as e:
        print(f"❌ [항목코드 요청 실패] 코드: {stat_code}, 응답: {res.text}")
        return []
    return data.get("StatItemList", {}).get("row", [])

# 전체 통계표 목록 불러오기
all_tables = get_stat_list()
if not all_tables:
    print("📛 통계표 목록이 비어있습니다. API 키 또는 응답 확인 필요.")
    exit()

# 키워드별 통계표 및 항목코드 출력
for key, keyword in search_keywords.items():
    print(f"\n🔍 [{key}] 관련 통계표 검색어: '{keyword}'")
    filtered = [t for t in all_tables if keyword in t.get("STAT_NAME", "")]
    if not filtered:
        print("❌ 관련 통계표 없음")
        continue

    for table in filtered:
        stat_code = table['STAT_CODE']
        print(f"📌 코드: {stat_code}, 이름: {table['STAT_NAME']}")
        items = get_item_list(stat_code)
        if not items:
            print("   └ ⚠️ 항목코드 없음")
            continue
        for i in items[:5]:  # 처음 5개만 출력
            print(f"   └ {i['ITEM_CODE']} : {i['ITEM_NAME']}")



🔍 [CPI] 관련 통계표 검색어: '소비자물가지수'
📌 코드: 0000000211, 이름: 4.2. 소비자물가지수(2020=100)
❌ [항목코드 요청 실패] 코드: 0000000211, 응답: <HTML>
<HEAD>
  	<TITLE>503 - Service Unavailable</TITLE>
</HEAD>

<!-- Background white, links blue (unvisited), navy (visited), red (active) -->
<BODY
  BGCOLOR="#FFFFFF"
  TEXT="#000000"
  LINK="#0000FF"
  VLINK="#000080"
  ALINK="#FF0000"
>

<BR><BR>
<H1 ALIGN="CENTER">
   	HTTP Error 503 - Service unavailable
</H1>

</BODY>
</HTML>
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      